<a href="https://colab.research.google.com/github/eeolga/article/blob/main/Formula_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Load Excel data
def load_excel_data(file_path):
    """
    Load Excel data into a Pandas DataFrame.
    """
    return pd.read_excel(file_path)

# Compute Cn = WiD + WnT + WnECTS
def compute_Cn(bloom_weights, tool_weights, tool_ects):
    """
    Compute Cn based on the given formula.
    """
    Cn = bloom_weights + tool_weights + tool_ects
    return Cn

# Compute WiD = i / sum(i)
def compute_WiD(i_values):
    """
    Compute WiD for i-th Bloom level weights.
    """
    total = sum(i_values)
    return np.array(i_values) / total

# Compute WnT = (1 / sum(T)) * n
def compute_WnT(n_values):
    """
    Compute WnT for n-th tool weights.
    """
    total = sum(n_values)
    return np.array(n_values) / total

# Compute WnECTS = (3 * factor / 100) * (sum(factors))
def compute_WnECTS(factors):
    """
    Compute WnECTS for n-th tool ECTS weights.
    """
    total_factor = sum(factors)
    return (3 * np.array(factors) / 100) * total_factor

# Compute DA = WA * fA, where fA = A * A
def compute_DA(WA, A):
    """
    Compute Dynamic Assessment component DA.
    """
    fA = np.array(A) * np.array(A)
    return WA * fA

# Main function to compute Competency Indicator (CA)
def compute_CA(data):
    """
    Compute Competency Indicator (CA) using the provided formula.

    Parameters:
        data: Pandas DataFrame with required input columns.

    Returns:
        DataFrame with CA values computed.
    """
    # Compute Bloom level weights WiD
    data['WiD'] = compute_WiD(data['Bloom_level'].tolist())

    # Compute Tool weights WnT
    data['WnT'] = compute_WnT(data['Tool_competence_indication'].tolist())

    # Compute Tool ECTS WnECTS
    data['WnECTS'] = compute_WnECTS(data['Tool_ECTS'].tolist())

    # Compute Cn = WiD + WnT + WnECTS
    data['Cn'] = compute_Cn(data['WiD'], data['WnT'], data['WnECTS'])

    # Compute DA = WA * fA, where fA = A ** A
    data['DA'] = compute_DA(data['Competency_Indicator_0.1'], data['Competency_Indicator_0.5'])

    # Compute CA = Cn + DA
    data['CA'] = data['Cn'] + data['DA']

    return data

# Example usage
if __name__ == "__main__":
    # Load data from an Excel file
    file_path = "/content/output 7-12-2023.xlsx"  # Replace with the actual file path
    data = load_excel_data(file_path)

    # Compute CA values
    result = compute_CA(data)

    # Save results back to an Excel file
    result.to_excel("computed_results.xlsx", index=False)
    print("Results saved to 'computed_results.xlsx'")


Results saved to 'computed_results.xlsx'


In [ ]:
import pandas as pd
import numpy as np

def calculate_weights(i_values, T_values, k_values):
    # Calculate WiD
    sum_i = sum(i_values)
    WiD = [i / sum_i for i in i_values]

    # Calculate WnT
    sum_T = sum(T_values)
    WnT = [(1 / sum_T) * n for n in T_values]

    # Calculate WnECTS
    # Convert the lists to NumPy arrays for element-wise multiplication
    k38_array = np.array(k_values["k38"])
    k28_array = np.array(k_values["k28"])
    k34_array = np.array(k_values["k34"])

    WnECTS = [(3 * 38 / 100 * k38_array + 3 * 28 / 100 * k28_array + 3 * 34 / 100 * k34_array)]
    WnECTS = WnECTS[0].tolist() # Convert back to list to maintain data structure consistency

    return WiD, WnT, WnECTS

def calculate_CA(Cn, DA):
    return Cn + DA

def calculate_DA(WA, fA):
    return WA * fA

def calculate_Cn(WiD, WnT, WnECTS):
    return sum(WiD) + sum(WnT) + sum(WnECTS)

def main(file_path):
    # Load data from Excel
    data = pd.read_excel(file_path)

    # Parse input columns
    i_values = data['Bloom_level_weight'].tolist()
    T_values = data['Tool_weight'].tolist()
    k_values = {
        "k38": data['k38'].tolist(),
        "k28": data['k28'].tolist(),
        "k34": data['k34'].tolist()
    }

    # Calculate weights
    WiD, WnT, WnECTS = calculate_weights(i_values, T_values, k_values)

    # Compute Cn
    Cn = calculate_Cn(WiD, WnT, WnECTS)

    # Compute DA
    WA = data['Assessment_0.1']
    A = data['Competency_Indicator_0.1']
    fA = A ** A
    DA = calculate_DA(WA, fA)

    # Compute CA
    CA = calculate_CA(Cn, DA)

    # Add results to the DataFrame
    data['WiD'] = WiD
    data['WnT'] = WnT
    data['WnECTS'] = WnECTS
    data['Cn'] = Cn
    data['DA'] = DA
    data['CA'] = CA

    # Error Analysis
    # Check if 'Expected_CA' column exists before calculating error
    if 'Expected_CA' in data.columns:
        data['Error'] = data['Expected_CA'] - data['CA']
    else:
        print("Warning: 'Expected_CA' column not found. Skipping error analysis.")
    # Save results back to Excel
    output_file = file_path.replace('.xlsx', '_results.xlsx')
    data.to_excel(output_file, index=False)

    print(f"Results saved to {output_file}")

if __name__ == "__main__":
    excel_file_path = "/content/output 12-12-2023.xlsx"
    main(excel_file_path)

Results saved to /content/output 12-12-2023_results.xlsx


In [ ]:
import pandas as pd
import numpy as np

def calculate_weights(i_values, T_values, k_values):
    # Normalize WiD
    sum_i = sum(i_values)
    WiD = [i / sum_i for i in i_values]

    # Normalize WnT
    sum_T = sum(T_values)
    WnT = [(1 / sum_T) * n for n in T_values]

    # Normalize WnECTS to sum to 3
    # Convert lists to NumPy arrays for element-wise multiplication
    WnECTS = [(3 / 100) * (38 * np.array(k_values["k38"]) + 28 * np.array(k_values["k28"]) + 34 * np.array(k_values["k34"]))]
    # Convert back to list to maintain data structure consistency
    WnECTS = WnECTS[0].tolist()

    # Ensure weights are scaled correctly
    WiD = [w / sum(WiD) for w in WiD]
    WnT = [w / sum(WnT) for w in WnT]
    WnECTS = [w * 3 / sum(WnECTS) for w in WnECTS]

    return WiD, WnT, WnECTS

def calculate_CA(Cn, DA):
    return Cn + DA

def calculate_DA(WA, fA):
    return WA * fA

def calculate_Cn(WiD, WnT, WnECTS):
    return sum(WiD) + sum(WnT) + sum(WnECTS)

def main(file_path):
    # Load data from Excel
    data = pd.read_excel(file_path)

    # Parse input columns
    i_values = data['Bloom_level_weight'].tolist()
    T_values = data['Tool_weight'].tolist()
    k_values = {
        "k38": data['k38'].tolist(),
        "k28": data['k28'].tolist(),
        "k34": data['k34'].tolist()
    }

    # Calculate weights
    WiD, WnT, WnECTS = calculate_weights(i_values, T_values, k_values)

    # Compute Cn
    Cn = calculate_Cn(WiD, WnT, WnECTS)

    # Compute DA
    WA = data['Assessment_0.1']
    A = data['Competency_Indicator_0.1']
    fA = A ** A
    DA = calculate_DA(WA, fA)

    # Compute CA
    CA = calculate_CA(Cn, DA)

    # Add results to the DataFrame
    data['WiD'] = WiD
    data['WnT'] = WnT
    data['WnECTS'] = WnECTS
    data['Cn'] = Cn
    data['DA'] = DA
    data['CA'] = CA

    # Error Analysis
    data['Error'] = data['Expected_CA'] - data['CA']

    # Rescale CA to align with Expected_CA if necessary
    scaling_factor = data['Expected_CA'].max() / data['CA'].max()
    data['CA_Scaled'] = data['CA'] * scaling_factor

    # Save results back to Excel
    output_file = file_path.replace('.xlsx', '_results_changed.xlsx')
    data.to_excel(output_file, index=False)

    print(f"Results saved to {output_file}")

if __name__ == "__main__":
    excel_file_path = "/content/output 12-12-2023.xlsx"  # Replace with the path to your Excel file
    main(excel_file_path)

Results saved to /content/output 12-12-2023_results_changed.xlsx


In [ ]:
import pandas as pd
import numpy as np

# Load data (replace 'your_excel_file.xlsx' with your actual file path)
data = pd.read_excel('/content/output 12-12-2023.xlsx')

# Define weights and parameters for the formulas
alpha = 0.3
beta = 0.2
gamma = 0.5
lambda_ = 0.7  # Importance of Cn in CA calculation
delta = 1.0  # Scaling factor for DA

# Calculate components
# WiD: Bloom level weight (normalized)
data['WiD'] = data['Bloom_level_weight']  # Assuming already normalized

# WnT: Tool weight (normalized)
data['WnT'] = data['Tool_weight']  # Assuming already normalized

# WnECTS: Adjusted Tool ECTS weight
data['WnECTS'] = (3 * (data['k38'] + data['k28'] + data['k34']) / 100) * (
data['k38'] + data['k28'] + data['k34']
)

# Cn: Tool Competence Indication with weighted components
data['Cn'] = alpha * data['WiD'] + beta * data['WnT'] + gamma * data['WnECTS']

# fA: Dynamic component factor (normalized A)
data['fA'] = data['Competence'] / data['Competence'].max()

# DA: Dynamic Component Assessment
data['DA'] = delta * data['Assessment_0.1'] * data['fA']

# CA: Competency Indicator (combined formula)
data['CA'] = lambda_ * data['Cn'] + (1 - lambda_) * data['DA']

# Scale CA to match Expected_CA
data['CA_Scaled'] = (data['CA'] / data['CA'].max()) * data['Expected_CA']

# Error: Difference between calculated and expected values
data['Error'] = data['CA_Scaled'] - data['Expected_CA']

# Save results to a new Excel file
data.to_excel('refined_results.xlsx', index=False)

print("Refined CA calculation complete. Results saved to 'refined_results.xlsx'.")

Refined CA calculation complete. Results saved to 'refined_results.xlsx'.
